# Predicting which Canadian province will have the highest percentage of COVID-19 recoveries in the last 3 months of 2021
### Group Members
Jason Ji (69217222), Mikayla Chau (29043288), Linda Zhang (95711800), Alireza Mohammadi (20700985)

In [2]:
set.seed(36)
library(tidyverse)
library(repr)
library(tidymodels)
options(repr.matrix.max.rows = 6)

── Attaching packages ─────────────────────────────────────── tidyverse 1.3.1 ──

✔ ggplot2 3.3.6     ✔ purrr   0.3.4
✔ tibble  3.1.7     ✔ dplyr   1.0.9
✔ tidyr   1.2.0     ✔ stringr 1.4.0
✔ readr   2.1.2     ✔ forcats 0.5.1

── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()

── Attaching packages ────────────────────────────────────── tidymodels 1.0.0 ──

✔ broom        1.0.0     ✔ rsample      1.0.0
✔ dials        1.0.0     ✔ tune         1.0.0
✔ infer        1.0.2     ✔ workflows    1.0.0
✔ modeldata    1.0.0     ✔ workflowsets 1.0.0
✔ parsnip      1.0.0     ✔ yardstick    1.0.0
✔ recipes      1.0.1     

── Conflicts ───────────────────────────────────────── tidymodels_conflicts() ──
✖ scales::discard() masks purrr::discard()
✖ dplyr::filter()   masks stats::filter()
✖ recipes::fixed()  masks stringr::fixed()
✖ dplyr::lag()      masks stats::lag()
✖ yardstick::spec() masks r

### Intoduction

keeping columns: date, province, comfirmed, probable cases, deaths, total cases, percent today, percent recovered
predictors: date, comfirmed, probable cases, deaths, total cases, percent today

### Preliminary Data Analysis

The data is mostly tidy, however there are many columns we do not need for our analysis. Tidying the data included taking out these unwanted columns, any unwanted rows (Canada and Repatriated Travellers), and changing some column names to make it more reader-friendly. 

In [3]:
covid <- read_csv("canada-covid-19.csv")
covid

Rows: 9762 Columns: 40
── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
chr   (3): prname, prnameFR, percentrecover
dbl  (36): pruid, update, numconf, numprob, numdeaths, numtotal, numtested, ...
date  (1): date

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.


pruid,prname,prnameFR,date,update,numconf,numprob,numdeaths,numtotal,numtested,⋯,ratedeaths_last14,numtotal_last7,ratetotal_last7,numdeaths_last7,ratedeaths_last7,avgtotal_last7,avgincidence_last7,avgdeaths_last7,avgratedeaths_last7,raterecovered
<dbl>,<chr>,<chr>,<date>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
35,Ontario,Ontario,2020-01-31,NA,3,0,0,3,NA,⋯,NA,NA,NA,NA,NA,NA,NA,NA,NA,0
59,British Columbia,Colombie-Britannique,2020-01-31,NA,1,0,0,1,NA,⋯,NA,NA,NA,NA,NA,NA,NA,NA,NA,0
1,Canada,Canada,2020-01-31,NA,4,0,0,4,NA,⋯,NA,NA,NA,NA,NA,NA,NA,NA,NA,0
⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋱,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮
62,Nunavut,Nunavut,2021-12-17,1,676,NA,4,676,NA,⋯,0.00,0,0.00,0,0.00,0,0.00,0,0.00,0.00
99,Repatriated travellers,Voyageurs rapatriés,2021-12-17,NA,13,NA,0,13,NA,⋯,NA,0,NA,0,NA,0,NA,0,NA,NA
1,Canada,Canada,2021-12-17,NA,1866907,NA,30032,1866907,NA,⋯,0.72,39216,102.54,132,0.35,5602,14.65,19,0.05,10.85


In [4]:
tidy_covid <- covid |> 
    mutate(province = prname) |>
    select(province, date, numconf, numprob, numdeaths, numtotal, percentrecover) |>
    filter(province != "Repatriated travellers") |>
    filter(province != "Canada") |>
    group_by(province)
tidy_covid

province,date,numconf,numprob,numdeaths,numtotal,percentrecover
<chr>,<date>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>
Ontario,2020-01-31,3,0,0,3,NA
British Columbia,2020-01-31,1,0,0,1,NA
Ontario,2020-02-08,3,0,0,3,NA
⋮,⋮,⋮,⋮,⋮,⋮,⋮
Yukon,2021-12-17,1621,NA,14,1621,96.79
Northwest Territories,2021-12-17,2082,NA,12,2082,98.75
Nunavut,2021-12-17,676,NA,4,676,99.26
